# Use Azure-SQL as a Key-Value Store

There are no specific features for create a key-value store solution, as Memory-Optimized Tables can be used very efficiently for this kind of workload.
Memory-Optimized tables can be configured to be Durable or Non-Durable. In the latter case data is never persisted and they really behave like an in-memory cache. Here's some useful links:
- [In-Memory OLTP in Azure SQL Database](https://azure.microsoft.com/en-us/blog/in-memory-oltp-in-azure-sql-database/)
- [Transact-SQL Support for In-Memory OLTP](https://docs.microsoft.com/en-us/sql/relational-databases/in-memory-oltp/transact-sql-support-for-in-memory-oltp)
- [Optimize performance by using in-memory technologies](https://docs.microsoft.com/en-us/azure/azure-sql/in-memory-oltp-overview)


## Create Key-Value Store Table

Memory-Optimized Table are actually *compiled* into .DLL for maximum performances, and they use a lock-free strategy to guarantee consistency and isolation


In [31]:
if (schema_id('cache') is null)
    exec('create schema [cache];')
go
create table [cache].MemoryStore
(
	[key] bigint not null,
	[value] nvarchar(max) null,
	index IX_Hash_Key unique hash ([key]) with (bucket_count= 100000)  
) with (memory_optimized = on,  durability = schema_only);
go

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.500

# Test Select / Insert / Update performances

To test performance of In-Memory table we want to minimize or even zero-out all other external dependecies to be able only to focus on pure Azure SQL performance. For this reason it is useful to create a Stored Procedure that using the Native Compilation (so that it will be compiled into a .DLL too), will try to execute 100K SELECT/UPSERT operations as fast as possibile. To simulate a more reaslistic worjload, it will get the cached JSON document, update it and then put it back into the In-Memory table acting as Key-Value store cache.

In [32]:
create or alter procedure cache.[Test]
with native_compilation, schemabinding
as 
begin atomic with (transaction isolation level = snapshot, language = N'us_english')	
    declare @i int = 0;
    declare @o int = 0;
	while (@i < 100000)
	begin
		declare @r int = cast(rand() * 100000 as int)
	
		declare @v nvarchar(max) = (select top(1) [value] from [cache].[MemoryStore] where [key]=@r);
        set @o += 1;
	
		if (@v is not null) begin
			declare @c int = cast(json_value(@v, '$.counter') as int) + 1;
			update [cache].[MemoryStore] set [value] = json_modify(@v, '$.counter', @c) where [key] = @r
            set @o += 1;
        end else begin
			declare @value nvarchar(max) = '{"value": "' + cast(sysdatetime() as nvarchar(max)) + '", "counter": 1}'
			insert into [cache].[MemoryStore] values (@r, @value)
            set @o += 1;
		end	

		set @i += 1;
	end
    select total_operations = @o;
end
go

Commands completed successfully.

Total execution time: 00:00:00.609


Run the procedure to execute the test, while taking notice of start and end time to calculate elapsed milliseconds. Remember, __100K iterations__ will be executed.

In [34]:
delete from [cache].[MemoryStore]
go
declare @s datetime2, @e datetime2;
set @s = sysutcdatetime();
exec cache.[Test];
set @e = sysutcdatetime();
select slo = databasepropertyex(db_name(), 'ServiceObjective'), elapsed_msec = datediff(millisecond, @s, @e)
go


(63320 rows affected)

(1 row affected)

Total execution time: 00:00:03.070

total_operations
200000


slo,elapsed_msec
BC_Gen5_4,2922


In something around __2.9__ seconds a 4 vCore Azure SQL database has been able to complete 100K iterations. As every iteration executed a SELECT followed by an INSERT or UPDATE, it means that 200K operations has been executed in 2.9 seconds, which in turn results to close to __69K query/sec__ !

## Cleanup

In [29]:
drop procedure [cache].[Test]
drop table [cache].[MemoryStore]
go

: Msg 3729, Level 16, State 1, Line 2
Cannot DROP TABLE 'cache.MemoryStore' because it is being referenced by object 'Get'.

Total execution time: 00:00:00.031